In [1]:
import math
import pandas as pd
import yaml

In [2]:
with open("data.yaml") as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as e:
        print(e)

card_df = pd.DataFrame(data["cardData"])
card_df["adjustedOdds"] = card_df["odds"] * 6 ** (4 - card_df["numDice"]) // 6
card_df["adjustedOdds"] = card_df["adjustedOdds"].apply(lambda x: round(x / 2.16, 2))
card_df["money"] = card_df["adjustedOdds"].apply(lambda x: math.floor(math.log2(360 / x))) + card_df["numDice"]

fish_df = pd.DataFrame(data["fishData"])

In [3]:
card_df

,reputation,description,numDice,odds,adjustedOdds,money
0,1,Roll any #,1,6,100.00,2
1,1,Roll a # >=3,1,4,66.67,3
2,1,Roll a # <=4,1,4,66.67,3
3,1,Roll a # <=3,1,3,50.00,3
4,1,Roll a # >=4,1,3,50.00,3
...,...,...,...,...,...,...
94,6,Roll three #s that sum to 14,3,15,6.94,8
95,6,Roll three #s that sum to 15,3,10,4.63,9
96,7,Roll three matching #s,3,6,2.78,10
97,8,Roll four different #s,4,360,27.78,7


In [4]:
fish_df

,name,scientificName,habitat,family,rarity
0,Pacific Barracuda,Sphyraena argentea,coast,barracuda,common
1,Pacific Tomcod,Microgadus proximus,coast,cod,uncommon
2,California Corbina,Menticirrhus undulatus,coast,croaker,common
3,Queenfish,Seriphus politus,coast,croaker,common
4,Shortfin Corvina,Cynoscion parvipinnis,coast,croaker,common
...,...,...,...,...,...
265,Rainbow Trout,Oncorhynchus mykiss,river,trout,common
266,Arctic Grayling,Thymallus arcticus,river,trout,uncommon
267,Lake Trout,Salvelinus namaycush,river,trout,uncommon
268,Cutthroat Trout,Oncorhynchus clarkii,river,trout,uncommon/rare


In [5]:
fish_df[fish_df.isnull().any(axis=1)]

,name,scientificName,habitat,family,rarity


In [6]:
card_df["numDice"].value_counts().sort_index().reset_index()

,numDice,count
0,1,28
1,2,49
2,3,20
3,4,2


In [7]:
pd.crosstab(card_df["reputation"], card_df["money"], margins=True)

money,2,3,4,5,6,7,8,9,10,13,All
reputation,,,,,,,,,,,
1,1,6,0,0,0,0,0,0,0,0,7
2,0,0,15,0,0,0,0,0,0,0,15
3,0,0,0,6,0,0,0,0,0,0,6
4,0,1,0,13,6,0,0,0,0,0,20
5,0,0,0,0,10,4,15,0,0,0,29
6,0,0,1,1,2,8,5,2,0,0,19
7,0,0,0,0,0,0,0,0,1,0,1
8,0,0,0,0,0,1,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,1,1


In [8]:
card_df.groupby(["numDice", "reputation", "money", "adjustedOdds"]).size().reset_index(name="count").sort_values(["numDice", "adjustedOdds"], ascending=[True, False]).reset_index(drop=True)

,numDice,reputation,money,adjustedOdds,count
0,1,1,2,100.00,1
1,1,1,3,66.67,2
2,1,1,3,50.00,4
3,1,2,4,33.33,15
4,1,3,5,16.67,6
5,2,4,3,100.00,1
6,2,4,5,44.44,2
7,2,4,5,30.56,6
8,2,4,5,27.78,1
9,2,4,5,25.00,4


In [9]:
pd.crosstab(fish_df["family"], fish_df["habitat"], margins=True)

habitat,coast,ocean,river,All
family,,,,
barracuda,1,0,0,1
billfish,0,5,0,5
carp,0,0,4,4
catfish,0,0,7,7
cod,1,5,0,6
croaker,7,1,0,8
dolphinfish,0,1,0,1
flatfish,17,4,0,21
flyingfish,0,1,0,1


In [10]:
pd.crosstab(fish_df["rarity"], fish_df["habitat"], margins=True)

habitat,coast,ocean,river,All
rarity,,,,
common,79,28,40,147
common/uncommon,37,3,2,42
rare,1,9,2,12
uncommon,14,21,14,49
uncommon/rare,5,6,9,20
All,136,67,67,270


In [11]:
coast_card_df = card_df[card_df["numDice"] <= 3]
ocean_card_df = card_df[card_df["numDice"] >= 3]
river_card_df = card_df[card_df["numDice"] <= 2]

coast_fish_df = fish_df[fish_df["habitat"] == "coast"].reset_index(drop=True)
ocean_fish_df = fish_df[fish_df["habitat"] == "ocean"].reset_index(drop=True)
river_fish_df = fish_df[fish_df["habitat"] == "river"].reset_index(drop=True)

coast_card_df = coast_card_df.sample(n=len(coast_fish_df), replace=True).reset_index(drop=True)
ocean_card_df = ocean_card_df.sample(n=len(ocean_fish_df), replace=True).reset_index(drop=True)
river_card_df = river_card_df.sample(n=len(river_fish_df), replace=True).reset_index(drop=True)

coast_df = coast_fish_df.join(coast_card_df, how="inner")
ocean_df = ocean_fish_df.join(ocean_card_df, how="inner")
river_df = river_fish_df.join(river_card_df, how="inner")

df = pd.concat([coast_df, ocean_df, river_df])
df

,name,scientificName,habitat,family,rarity,reputation,description,numDice,odds,adjustedOdds,money
0,Pacific Barracuda,Sphyraena argentea,coast,barracuda,common,5,Roll two #s that sum to 7,2,6,16.67,6
1,Pacific Tomcod,Microgadus proximus,coast,cod,uncommon,3,Roll a 1,1,1,16.67,5
2,California Corbina,Menticirrhus undulatus,coast,croaker,common,5,Roll two #s that sum to 7,2,6,16.67,6
3,Queenfish,Seriphus politus,coast,croaker,common,4,Roll a 1 and a # >=4,2,6,16.67,6
4,Shortfin Corvina,Cynoscion parvipinnis,coast,croaker,common,2,Roll a 1 or 4,1,2,33.33,4
...,...,...,...,...,...,...,...,...,...,...,...
62,Rainbow Trout,Oncorhynchus mykiss,river,trout,common,3,Roll a 1,1,1,16.67,5
63,Arctic Grayling,Thymallus arcticus,river,trout,uncommon,5,Roll two #s that sum to 5,2,4,11.11,7
64,Lake Trout,Salvelinus namaycush,river,trout,uncommon,5,Roll a 2 and 4,2,2,5.56,8
65,Cutthroat Trout,Oncorhynchus clarkii,river,trout,uncommon/rare,2,Roll a 2 or 6,1,2,33.33,4


In [12]:
with open("merged_output.yaml", "w") as file:
    data = df.drop(columns=["numDice", "odds", "adjustedOdds", "scientificName"]).to_dict(orient="records")
    yaml.dump({"fish": data}, file, sort_keys=False, indent=2)

In [13]:
sorted_df = fish_df.sort_values(by=["habitat", "family", "rarity", "name"])

markdown_lines = [
    "| name | scientificName | habitat | family | rarity |",
    "|------|----------------|---------|--------|--------|"
]

for _, row in sorted_df.iterrows():
    markdown_lines.append(
        f"| {row['name']} | {row['scientificName']} | {row['habitat']} | "
        f"{row['family']} | {row['rarity']} |"
    )

with open("fish_table.md", "w") as file:
    file.write("\n".join(markdown_lines))